In [1]:
import plotly.express as px
import plotly

import mysql.connector
from db_creds import host, user, passwd, database

import pandas as pd

# connecting to database
db = mysql.connector.connect(
    host=host,
    user=user,
    passwd=passwd,
    database=database
)

In [2]:
# winrates
winrate_df = pd.read_sql('''
SELECT DISTINCT
	c.name AS champion,
    sum(gi.Win)/count(gi.Win) AS winrate,
    1 - sum(gi.Win)/count(gi.Win) AS loserate
FROM game_instance gi
	INNER JOIN champ_select cs ON gi.SummonerID = cs.SummonerID
	INNER JOIN game_champ gc ON gi.GameID = gc.GameID AND gc.ChampionID = cs.ChampionID
    INNER JOIN champion c on gc.ChampionID = c.ID
GROUP BY
	c.name
''', con=db).set_index('champion')

In [3]:
# pickrates
pickrate_df = pd.read_sql('''
SELECT DISTINCT
	c.name AS champion,
    (COUNT(ChampionID) OVER (PARTITION BY ChampionID))/(SELECT COUNT(*) FROM game) AS pickrate,
    1 - (COUNT(ChampionID) OVER (PARTITION BY ChampionID))/(SELECT COUNT(*) FROM game) AS nopickrate
FROM game_champ gc
	INNER JOIN champion c ON gc.ChampionID = c.Id
''', con=db).set_index('champion')

In [8]:
# damage dealt
damage_df = pd.read_sql('''
SELECT DISTINCT
	c.name AS champion,
	avg(gi.TotalDamageDealtToChampions) as avg_damage_per_game
FROM game_instance gi
	INNER JOIN champ_select cs ON gi.SummonerID = cs.SummonerID
	INNER JOIN game_champ gc ON gi.GameID = gc.GameID AND gc.ChampionID = cs.ChampionID
    INNER JOIN champion c on gc.ChampionID = c.ID
GROUP BY
	c.name
ORDER BY
	avg_damage_per_game
''', con=db)

In [9]:
damage_df.head()

,champion,avg_damage_per_game
0,Taric,5655.8434
1,Soraka,5894.3206
2,Lulu,5999.3915
3,Rakan,6844.3455
4,Braum,7228.8417


In [16]:
winrate_df = winrate_df.T
pickrate_df = pickrate_df.T

In [17]:
pickrate_df['Annie']

pickrate      0.0384
nopickrate    0.9616
Name: Annie, dtype: float64

In [18]:
# pie chart for winrate
winrate_fig = px.pie(
    winrate_df,
    names=winrate_df.index,
    values=winrate_df['Annie']
)

In [19]:
# pie chart for pickrate
pickrate_fig = px.pie(
    pickrate_df,
    names=pickrate_df.index,
    values=pickrate_df['Annie']
)

In [10]:
# bar chart for damage dealt
dmg_fig = px.bar(damage_df, x='champion', y='avg_damage_per_game')

In [11]:
# plotly.offline.plot(pickrate_fig,filename='Piechart.html')
plotly.offline.plot(dmg_fig,filename='Barchart.html')

'Barchart.html'